#準備
GitHubに上がっている写真、事例用csvファイルをGoogle Colabにアップロードする。

# ngrokへの登録
Streamlit を Google Colab で使用するには、「ngrok」の利用が必須になる。


In [12]:
# ngrokを設定
NGROK_AUTH_TOKEN = "Your AuthoTaken"

#校正コード

In [2]:
#ここで使うものをinstallしておく
!pip install streamlit # streamlit
!pip install PyPDF2 # PDF読み込み
!pip install torch torchvision # モデル関連
!pip install pytorch-lightning # モデル関連
!pip install transformers # モデル関連
!pip install pyngrok # streamlit関連

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/

In [4]:
%%writefile back.py
import os
import torch
import time
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

# プロンプトの作成
def make_q_all(examples):
    category_groups = {}
    for example in examples:
        category = example["種類"]
        if category not in category_groups:
            category_groups[category] = []
        category_groups[category].append(f"校正前：{example['校正前']} \n校正後：{example['校正後']}")

    formatted_prompt = "これから、日本語の観光雑誌を校正してもらいます。以下の説明を参考にして、文章校正をしてください。\n\n"
    formatted_prompt += '''1. 表記揺れに関する校正
状況に応じて、漢字から平仮名、または平仮名から漢字の変換をしてください。ただし、画数が多い漢字や中学生以降に習う難しい漢字は避けてください。
また、数値表現の単位は国際単位系のSI単位に校正してください。SIに含まれない単位はその名称に校正してください。"A県・B町"という形式は、"A県・B"という形式で出力してください。

2. 記号に関する校正
句読点や中点を適切に追加・削除してください。

3. 誤りに関する校正
誤字脱字を訂正してください。

4. 表現に関する校正
話し言葉は書き言葉に直してください。
店名や商品名には「」を付けて強調してください。ただし、「」内に助詞が含まれる場合は「」を削除してください。

その他、必要な校正は適切に施してください。
'''
    for category, corrections in category_groups.items():
        formatted_prompt += "\n"  # カテゴリー間の区切り
        formatted_prompt += "\n".join(corrections)  # 校正例

    formatted_prompt += "\n\nそれでは上の説明を参考に、次の文を校正してください。この時、校正後の文のみを出力してください。\n"
    formatted_prompt += "校正前：{inputs}\n校正後："

    return formatted_prompt

# モデルのロード
def load_model():
    model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.2"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )

    model.generation_config.temperature = None
    model.generation_config.top_p = None

    return tokenizer, model

# 校正を行う関数
def proofread(input_file="origin.txt", output_file="corrected.txt", tokenizer=None, model=None):
    if tokenizer is None or model is None:
        raise ValueError("TokenizerとModelがロードされていません。")

    start = time.time()

    # 校正する文章を読み込み
    with open(input_file, "r", encoding="utf-8") as f:
        sentences = f.readlines()
        print(sentences)

    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"

    # 事例データ読み込み
    shots = pd.read_csv("/content/Proofreading_data_train.csv", encoding='utf-8')
    examples = [{"校正前": row['修正前'], "校正後": row['修正後'], "種類": row['カテゴリー']} for _, row in shots.iterrows()]
    print(f'事例データ: {(examples)}')

    # プロンプト作成
    query = make_q_all(examples)

    # 校正結果を格納するリスト
    results = []

    start = time.time()
    for sentence in sentences:
        sentence = sentence.strip()
        if sentence == "":
            continue  # 空行はスキップ

        messages = [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": query.format(inputs=sentence)}
        ]

        # トークナイズ
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        # 終了トークン
        terminators = [
            tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        # 推論を実行
        outputs = model.generate(
            input_ids,
            max_new_tokens=512,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
        )

        # 出力のデコード
        response = outputs[0][input_ids.shape[-1]:]
        corrected_sentence = tokenizer.decode(response, skip_special_tokens=True).strip()

        results.append(corrected_sentence)

    end = time.time()
    print(f"校正にかかった時間: {end - start:.2f}秒")

    # 校正結果をファイルに保存
    corrected_text = "\n".join(results)
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(corrected_text)

    return corrected_text

# モデルをロード（フロントエンドで再ロードを防ぐため）
# tokenizer, model = load_model()
tokenizer, model = load_model()
corrected_text = proofread("origin.txt", "corrected.txt", tokenizer, model) # 校正関数を呼び出す



Writing back.py


In [6]:
# フロントエンド
%%writefile app.py

import streamlit as st
import PyPDF2

# 校正関数
import back

# モデルをロード（アプリ起動時に1度だけ）
tokenizer, model = back.load_model()

def main():
  st.title("文章校正アプリ")

  if st.button("使い方"):
      # page1.pyに切り替える。なお、サブページは必ずpagesディレクトリに配置する必要がある
      st.switch_page("pages/How_To_Use.py")

  # PDFファイルのアップロード
  st.text("校正したいファイルをアップロードしてください。")
  uploaded_file = st.file_uploader("PDFファイルをアップロードしてください。", type=["pdf"])

  # PDFからテキストを抽出する関数
  def extract_text_from_pdf(file):
      reader = PyPDF2.PdfReader(file)
      text = ''
      for page in reader.pages:
          text += page.extract_text()
      return text

  if uploaded_file is not None:
      # PDFファイルをバイナリモードで読み込む
      with uploaded_file as file:
         before_text = extract_text_from_pdf(file)
      if before_text is None:
        st.warning("テキストが抽出できませんでした。別のPDFを試してみてください。")

      if st.button("校正する"):
          # 抽出されたテキストを1行にしてテキストファイルに書き込みます
          # origin.txtがPDFから読み取ったファイル
          # replaceで文章の形を整える
          with open("origin.txt", "w", encoding="utf-8") as file:
              before_text = before_text.replace("\n", "")
              before_text = before_text.replace("。", "。\n")
              file.write(before_text)

          # ファイルを使って、校正を行う
          #corrected_text = back.proofread("origin.txt", "corrected.txt", tokenizer, model) # 校正関数を呼び出す

          # 校正結果を保存
          #with open("corrected.txt", "w", encoding="utf-8") as file:
           #   file.write(corrected_text)


          # 校正結果を表示するshow.pyに切り替える。なお、サブページは必ずpagesディレクトリに配置する必要がある
          st.switch_page("pages/show.py")



if __name__ == '__main__':
    main()

Writing app.py


In [7]:
# ディレクトリの作成
!mkdir pages

In [14]:
# 使い方説明ページ

%%writefile pages/How_To_Use.py

import streamlit as st
from PIL import Image

st.title("使い方")

st.text("1.「Browse file」ボタンからファイルをアップロードします。")

# 画像ファイルの絶対パスを指定
img_path = r"/content/home.png"
img = Image.open(img_path)
st.image(img)

st.text("2.[校正する]ボタンをクリックします。（実行には時間がかかります。）")
img_path = r"/content/check.png"
img = Image.open(img_path)
st.image(img)

st.text("3.校正されたテキストが表示されます。")
img_path = r"/content/show.png"
img = Image.open(img_path)
st.image(img)


Overwriting pages/How_To_Use.py


In [9]:
# 結果の表示
%%writefile pages/show.py

import streamlit as st
import difflib
import pandas as pd

# ファイルの表示
# before.txt の内容をStreamlitで表示
with open("origin.txt", "r", encoding='utf-8') as f:
    before_content = f.read()

# after.txt の内容をStreamlitで表示
with open("corrected.txt", "r", encoding='utf-8') as f:
    after_content = f.read()

# Streamlitで表示
#st.subheader("校正前")
#st.text(before_content)
#st.subheader("校正後")
#st.text(after_content)  # または st.write(after_content) でも表示可能

res = difflib.ndiff(before_content.split(), after_content.split())

st.subheader("校正前")
for r in res:
    if r[0] == "-":
        st.text(r[1:])

res = difflib.ndiff(before_content.split(), after_content.split())

st.subheader("校正後")
for r in res:
    if r[0] == '+':
        st.text(r[1:])

Writing pages/show.py


In [19]:
from pyngrok import ngrok,conf

# taken = "登録したngrokのAPI"に変更する
token = NGROK_AUTH_TOKEN
!pkill -f streamlit
!streamlit run app.py --server.port 8501 &>/dev/null &
!pgrep streamlit

!pkill ngrok

ngrok.set_auth_token(token)

#Streamlitが使用するデフォルトのポート番号8501を明示的に指定
public_url = ngrok.connect("8501").public_url
def convert_http_to_https(url):
    # urlの先頭がhttpだったらhttpsに変更
    if url.startswith("http://"):
        # 1は置換する回数(1を指定することで、元のURL内で最初に見つかった "http://" の部分だけを "https://" に置き換え、他の "http://" は変更されない。)
        url = url.replace("http://", "https://", 1)
    return url
print(convert_http_to_https(public_url))

9219
https://e4f8-35-199-146-168.ngrok-free.app


数字が一つとURLが一つ表示されれば、問題なく実行できています。

URLをクリックすることでWebアプリを開くことができます。

数字が二つ、errorという表示などがあれば、このセルのみを何度か実行してください。